<h1><b>Statistique en Bioinformatique : </b> TME 4  et 5 </h1>
<br>
L’objectif de ce TME est: 
<br>
<ul>
<li> implémenter l'algorithme de Viterbi et l'estimation des paramètres (en utilisant le Viterbi training)
pour l'exemple du occasionally dishonest casino.   </li> 
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><h3>Soumission</h3></p>
<ul>
<li>Renomer le fichier TME4_5.ipynb pour TME4_5_NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre à  https://moodle-sciences.upmc.fr/moodle-2019/course/view.php?id=4664</li>
</ul>
</div>
</div>

</div>

Nom etudiant 1 :
<br>
Nom etudiant 2 :
<br>

<h3>Introduction</h3>
Un casino parfois malhonnête (occasionally dishonest casino) utilise 2 types de pieces : fair et unfair. <br>
La matrice de transition entre les états cachés est:<br>
${\cal S}=\{F,U\}$ (fair, unfair):
$$
p = \left(
\begin{array}{cc}
0.99 & 0.01\\
0.05 & 0.95
\end{array}
\right)\ ,
$$

les probabilités d'éemission des symboles 
${\cal O} = \{H,T\}$ (head, tail):
\begin{eqnarray}
e_F(H) =  0.5 &\ \ \ \ &
e_F(T) = 0.5 \nonumber\\
e_U(H) = 0.9 &\ \ \ \ &
e_U(T) = 0.1 \nonumber
\end{eqnarray}

<br> Et la condition initiale $\pi^{(0)} = (1,0)$ (le jeux commence toujours avec le pieces juste (fair).

<b>Exercice 1</b>:
<u>Simulation</u>: Ecrire une fonction qui simule $T$ jets de pieces. 
La fonction renverra un tableau à deux colonnes correspondant 
aux valeurs simulées pour les états cachés $X_t$ 
(type de dés utilisée, “F” ou “U”) et aux symboles observées $Y_t$ 
(résultat du jet de dés, “H” ou “T”). On simulera une séquence
de longueur 2000 qu'on gardera pour les applications ultérieures.


In [2]:
import numpy as np
import matplotlib.pyplot as plt


S = { 0:'F',1 :'U'}
Pij = np.array([[0.99,0.01], [0.05,0.95]])

O = {0:'H', 1: 'T'}
Eij = np.array([[0.5,0.5], [0.9,0.1]]) #ça aurait dû être Eio

# Condition initiale
pi0=np.array([0.999,0.001])

T = 2000

In [3]:
import random

def lancer(Pi,M,E,S,O,T):
    M_util=np.cumsum(M,axis=1)
    Pi_util=np.cumsum(Pi)
    E_util=np.cumsum(E,axis=1)
    Tab=np.zeros((T,2))

    #print(M_util,'\n')
    #print(Pi_util,'\n')
    #print(E_util,'\n')
    
    
   # Creation du Tableau en int 
    
    #Première itération:
    
    Choix_etat=random.random()
    Choix_observation=random.random()
    
    etat=np.where(Choix_etat>=Pi_util)[0]
    etat=len(etat)
    
    Tab[0][0]=etat
    
    observation=np.where(Choix_observation>=E_util[etat])[0]
    observation=len(observation)
    
    Tab[0][1]=observation
    
    #Suite du tableau:
    
    for i in range(1,T):
        
        Choix_etat=random.random()
        Choix_observation=random.random()
    
        etat=np.where(Choix_etat>=M_util[int(Tab[i-1][0])])[0]
        etat=len(etat)
        
        Tab[i][0]=etat
        
        observation=np.where(Choix_observation>=E_util[etat])[0]
        observation=len(observation)
    
        Tab[i][1]=observation
  #Conversion du tableau en str
    Tab2=np.empty((T,2),dtype=str)
    for i in range(len(Tab)):
        Tab2[i][0]=S[int(Tab[i][0])]
        Tab2[i][1]=O[int(Tab[i][1])]
            
            
    return Tab,Tab2
# Fonction qui simule T jets de pieces


L2000_int,L2000=lancer(pi0,Pij,Eij,S,O,2000)
print(L2000_int)
print(L2000)

[[0. 1.]
 [0. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 0.]]
[['F' 'T']
 ['F' 'H']
 ['F' 'T']
 ...
 ['F' 'T']
 ['F' 'T']
 ['F' 'H']]


<b>Exercice 2</b>: <u>Algorithme de Viterbi </u>: Ecrire une fonction qui permet
de déterminer la séquence $(i^\star_t)_{t=0:T}$ d'états cachés
plus probable, ansi que sa probabilité. Pour tester votre fonction utiliser le résultat de la 
simulation (2éme colonne) de la question 1. Comparer $(i^\star_t)_{t=0:T}$ avec
les vrais états cachés (1ère colonne de la simulation). 


In [63]:
# Algorithme de Viterbi

def viterbiAlgo(P,E,Pi,seqO,S):
    viterbi_value=np.ones((len(P),len(seqO)))
    viterbi_traceback=np.zeros((len(P),len(seqO)))
    
    
    #initialisation:
    for x in range(len(P)):
        viterbi_value[x][0]=np.log10(Pi[x]*E[x][int(seqO[0])])
        viterbi_traceback[x][0]=-1
    #Récurrence:
    for i in range(1,len(seqO)):
        for j in range(len(P)):
            choix=[]
            for k in range(len(P)):
                choix.append(np.log10(P[k][j])+viterbi_value[k][i-1])
            viterbi_value[j][i]=np.log10(E[j][int(seqO[i])])+np.max(choix)   
            viterbi_traceback[j][i]=np.argmax(choix)
            
    ind=np.argmax(viterbi_value[:][-1])
    Etats_caches=[]
    
    for i in range(len(viterbi_traceback[0])-1,-1,-1):
        Etats_caches.append(S[int(ind)])
        ind=viterbi_traceback[int(ind)][i]
    
    Etats_caches.reverse()
    
    return Etats_caches

Etats=viterbiAlgo(Pij,Eij,pi0,L2000_int[:,1],S)

print(lolz)

['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',

<b>Exercice 3</b>: <u>Estimation des paramètres</u>
<br>
3.1) Ecrire une fonction qui utilise tous les résultats de la simulation
(états et symboles) pour compter les nombres d'occurrence $N_{ij}$ est $M_{iO}$ définis
en cours. Estimer $p_{ij}$ est $e_i(O)$, voir slides  37-39 dans la presentation. Attention, pour eviter les probabilites à zero nous alons utiliser les pseudo-count.

In [ ]:
# Estimation de Parametres par contage


3.2) <u> Viterbi training </u>: Ecrire une fonction qui utilise 
seulement la séquence $(O_t)_{t=0:T}$ (2emme colone de la simulation) pour estimer les 
paramètres $p_{ij}$ est $e_i(O)$. On s'arretera quand les diferences entre les logVraissamblance est inferieur à 1e-04. Comparer les résultats de 3.1 et de 3.2 (3.2 avec plusieurs restarts,
et avec initialisation des paramètres alèatoire).


In [ ]:
import matplotlib.pyplot as plt

# Initialisation aleatoire de Pij, Eij, pi0


# Calcule log Vraissamblance


# Viterbi Training


3.3) <u>Viterbi training deuxiemme version </u> Ecrivez une version de 3.3 qui:
- part plusieurs fois (100x) d'une initialisation aléatoire des 
paramètres de l'HMM,
- utilise Viterbi training pour estimer les paramètres,
- calcule la log-vraisemblance pour les paramètres estimés,
- sauvegarde seulement l'estimation avec la valeur maximale de la
log-vraisemblance.

Qu'est-ce que vous observez?



In [ ]:
# Viterbi Training  deuxiemme version
